# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Estimated memory usage (per MPI process):
        nonlocal projectors  :   3.42MiB
        single ψ             :    486KiB
        single ρ             :    108KiB
        peak memory (SCF)    :   5.99MiB
    
    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        pseudopot. family    : Pse

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Estimated memory usage (per MPI process):
        nonlocal projectors  :   88.3MiB
        single ψ             :   8.83MiB
        single ρ             :    432KiB
        peak memory (SCF)    :    118MiB
    
    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        pseudopot. family    : Ps

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -9.355232160213                   -1.10    6.1    148ms
  2   -9.356788283921       -2.81       -1.43    1.0   77.9ms
  3   -9.357063948325       -3.56       -2.81    2.4   88.7ms
  4   -9.357110507166       -4.33       -3.01    8.1    205ms
  5   -9.357110903711       -6.40       -3.13    1.0   75.7ms
  6   -9.357111186762       -6.55       -3.27    2.4   87.7ms
  7   -9.357111377378       -6.72       -3.45    1.0   75.1ms
  8   -9.357111451608       -7.13       -3.61    1.0   74.0ms
  9   -9.357111491542       -7.40       -3.85    2.1   86.1ms
 10   -9.357111504641       -7.88       -4.07    1.2   79.1ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -18.74730940109                   -0.97    5.9    386ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     4
│     1
│     1
│     1
│     1
│     6
│     1
│     1
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  2   -18.75447462054       -2.14       -1.34    2.0    317ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│      6
│      2
│      4
│     12
│      4
│      9
│      1
│      8
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  3   -18.79218446501       -1.42       -1.89    5.8    352ms
  4   -18.79256597606       -3.42       -2.14    3.4    319ms
  5   -18.79259613109       -4.52       -3.29    2.0    687ms
  6   -18.79260484267       -5.06       -3.43    8.1    467ms
  7   -18.79260617192       -5.88       -3.72    1.0    192ms
  8   -18.

In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -37.54968271570                   -0.84    8.2    1.59s
  2   -37.55786728947       -2.09       -1.23    2.8    817ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     12
│      5
│     15
│     17
│     13
│     14
│     11
│     10
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  3   -37.56442008687       -2.18       -2.11   12.1    1.96s
  4   -37.56486331980       -3.35       -2.33   12.6    1.59s
  5   -37.56494159654       -4.11       -3.11    1.4    824ms
  6   -37.56494629497       -5.33       -3.47    8.8    1.54s
  7   -37.56494968718       -5.47       -3.98    3.1    931ms
  8   -37.56494986775       -6.74       -4.49    3.9    1.16s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -9.355045740077                   -1.10    6.0    307ms
  2   -9.356764115656       -2.76       -1.90    1.0    227ms
  3   -9.357007228685       -3.61       -2.25    4.5    128ms
  4   -9.357098679882       -4.04       -2.69    3.2   97.6ms
  5   -9.357111182055       -4.90       -3.67    1.5   72.5ms
  6   -9.357111426962       -6.61       -3.82    6.0    149ms
  7   -9.357111493124       -7.18       -4.17    1.1   66.3ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -37.54410109160                   -0.84    9.5    2.07s
  2   -37.55620847666       -1.92       -1.61    3.5    765ms
┌ Warning: Detected large minimal occupation 0.5748565253056509. SCF could be unstable. Try switching to adaptive band selection (`nbandsalg=AdaptiveBands(model)`) or request more converged bands than 29 (e.g. `nbandsalg=AdaptiveBands(model; n_bands_converge=32`)
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:87
  3   -35.65102935506   +    0.28       -0.92    8.4    1.40s
  4   -37.44870154057        0.25       -1.49    6.8    1.48s
  5   -37.54283517508       -1.03       -1.76    2.5    858ms
  6   -37.36735532959   +   -0.76       -1.41    5.1    1.13s
  7   -37.51160569524       -0.84       -1.68    4.9    1.13s
  8   -37.56206631876       -1.30       -2.10    2.5    1.30s
  9   -37.56144962584   +   -3.21       -2.06   

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.